---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2022</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1>Issued Thursday, 3/10/2022; Due Thursday, 3/24/2022</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Feel free to get help from others, but **the work you submit in should be your own.** Furthermore, this assignment may end up overlapping Problem Set #5, so plan accordingly. 

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Jenna"
    last = "Levy"
    onyen = "jenlevy"
    pid = "730280089"

The following problems make use of the following Insulin protein sequences

In [2]:
human = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
pig = "MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN"

In [10]:
from numpy import *

v = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
w = "MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN"

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

s, b = findLCS(human, pig)

def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LCS(b,v,i,j-1)
    else:
        return LCS(b,v,i-1,j)
        
print(LCS(b,v,b.shape[0]-1,b.shape[1]-1))

x = LCS(b,v,b.shape[0]-1,b.shape[1]-1)
print(len(x))

def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])


MALWRLLPLLALLALWPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKRREAEQGLQLALEGQKRGIVEQCCTSICSLYQLENYCN
87
v = MALWM_RLLPLLALLALWGPD_PA_A_AFVNQHLCGSHLVEALYLVCGERGFFYTPKT_RREAEDL__Q_GS__________LQP_LALEGSL__QKRGIVEQCCTSICSLYQLENYCN
w = MALW_TRLLPLLALLALW___APAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPK_ARREAE__NPQAG_AVELGGGLGGLQ_ALALEG__PPQKRGIVEQCCTSICSLYQLENYCN


---
**Problem #1:** Find the longest common subsequence (LCS) shared between human and pig insulin, its length, and the alignment that it implies.

In [19]:
import numpy

def GlobalAlign(v, w, match, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                continue
            if (v[i-1] == w[j-1]): 
                score = s[i-1,j-1] + match
            else:
                score = s[i-1,j-1] 
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s, b)

def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result



human = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
pig = "MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN"

s, b = GlobalAlign(human,pig, 1, -1)
print(s)
print("Best score =", s[-1,-1])
print(b)
align = Alignment(b,human,pig,b.shape[0]-1,b.shape[1]-1)
print("human =", align[0])
print("pig   =", align[1])




[[   0   -1   -2 ... -106 -107 -108]
 [  -1    1    0 ... -104 -105 -106]
 [  -2    0    2 ... -102 -103 -104]
 ...
 [ -96  -94  -92 ...   75   74   73]
 [ -97  -95  -93 ...   74   76   75]
 [ -98  -96  -94 ...   73   75   77]]
Best score = 77
[[0 2 2 ... 2 2 2]
 [1 3 2 ... 2 2 2]
 [1 1 3 ... 2 2 2]
 ...
 [1 1 1 ... 3 2 2]
 [1 1 1 ... 1 3 2]
 [1 1 1 ... 1 1 3]]
human = MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQ_GS__L_______QPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
pig   = MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN


---
**Problem #2:** Find an optimal *global alignment* between human and pig insulin assuming the following scoring from the "Alignment Game" described in Lecture 15, slide 4:

* +1 for a match
* 0 for a mismatch
* -1 for an indel

What is score for this alignment and the alignment itself? 

In [28]:
from numpy import *

v = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
w = "MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN"

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

s, b = findLCS(human, pig)

def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LCS(b,v,i,j-1)
    else:
        return LCS(b,v,i-1,j)
        
print(LCS(b,v,b.shape[0]-1,b.shape[1]-1))

x = LCS(b,v,b.shape[0]-1,b.shape[1]-1)
print(len(x))

def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

import numpy

def GlobalAlign(v, w, match, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    scores = 0
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                    scores+= indel
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                scores += indel
                continue
            if (v[i-1] == w[j-1]): 
                score = s[i-1,j-1] + match
                scores += match
            else:
                score = s[i-1,j-1] - 108
                scores -= 108
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s, b)

s_other, b_other = GlobalAlign(align[0],align[1], 1, -1)
print(s_other)
print("Best score =", s_other[-1,-1])
print(b_other)
print(scores)




MALWRLLPLLALLALWPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKRREAEQGLQLALEGQKRGIVEQCCTSICSLYQLENYCN
87
v = MALWM_RLLPLLALLALWGPD_PA_A_AFVNQHLCGSHLVEALYLVCGERGFFYTPKT_RREAEDL__Q_GS__________LQP_LALEGSL__QKRGIVEQCCTSICSLYQLENYCN
w = MALW_TRLLPLLALLALW___APAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPK_ARREAE__NPQAG_AVELGGGLGGLQ_ALALEG__PPQKRGIVEQCCTSICSLYQLENYCN
[[   0   -1   -2 ... -117 -118 -119]
 [  -1    1    0 ... -115 -116 -117]
 [  -2    0    2 ... -113 -114 -115]
 ...
 [-117 -115 -113 ...   48   47   46]
 [-118 -116 -114 ...   47   49   48]
 [-119 -117 -115 ...   46   48   50]]
Best score = 50
[[0 2 2 ... 2 2 2]
 [1 3 2 ... 2 2 2]
 [1 1 3 ... 2 2 2]
 ...
 [1 1 1 ... 3 2 2]
 [1 1 1 ... 1 3 2]
 [1 1 1 ... 1 1 3]]
-1411034


**Problem #3:** According to the rules of the "Alignment Game", what score does your alignment from **Problem 1** achieve? How would the rules of the alignment game need to be changed so that your solution for **Problem 1** would be the optimal answer?

In [20]:
from numpy import *

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

human = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
human_reverse = human[-1::-1]
sr, br = findLCS(human, human_reverse)


def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LCS(b,v,i,j-1)
    else:
        return LCS(b,v,i-1,j)
    

lcs_rev = LCS(br,human,br.shape[0]-1,br.shape[1]-1)
print(len(lcs_rev))
print(lcs_rev)
print(ord("M"))

33
LLLLALPQLSLEAERTKTREAELSLQPLALLLL
77


---
**Problem #4:** Find the length of the longest common subsequence of the human insulin sequence with the same sequence reversed.

In [11]:
import numpy

def GlobalAlign(v, w, match, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                continue
            ## if theres a match
            if (v[i-1] == w[j-1]): 
                score = s[i-1,j-1] + match
            ## if they're adjecent
            elif ((ord(v[i-1]) + 1) == ord(w[j-1]) or (ord(v[i-1]) - 1) == ord(w[j-1])):
                score = s[i-1,j-1] 
            else:
                score = s[i-1,j-1] - 1
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s, b)

def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

match = {('A','A'):  1, ('A','C'): -2, ('A','G'): -1, ('A','T'): -2,
         ('C','A'): -2, ('C','C'):  1, ('C','G'): -2, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -2, ('G','G'):  1, ('G','T'): -2,
         ('T','A'): -2, ('T','C'): -1, ('T','G'): -2, ('T','T'):  1}


human = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
human_reverse = human[-1::-1]

s, b = GlobalAlign(human,human_reverse, 1, -1)
print(s)
print("Best score =", s[-1,-1])
print(b)
align = Alignment(b,human,human_reverse,b.shape[0]-1,b.shape[1]-1)
print("human        =", align[0])
print("human_reverse=", align[1])

## spliting the human string before and after the constraint
human_pre = human[:human.index('EAE')]
human_suf = human[human.index('EAE'):]

##splitting the reverse string before and after the constraint
rev_pre = human_reverse[:human_reverse.index('EAE')]
rev_suf = human_reverse[human_reverse.index('EAE'):]

## two different allignments
s_pre, b_pre = GlobalAlign(human_pre,rev_pre, 1, -1)
pre_align = Alignment(b_pre, human_pre, rev_pre, b_pre.shape[0]-1, b_pre.shape[1]-1)

s_suf, b_suf = GlobalAlign(human_suf,rev_suf, 1, -1)
suf_align = Alignment(b_suf, human_suf, rev_suf, b_suf.shape[0]-1, b_suf.shape[1]-1)

## best score with constraints
print("Best score =", s_pre[-1,-1] + s_suf[-1,-1])


[[  0  -1  -2 ... -96 -97 -98]
 [ -1   0  -1 ... -94 -95 -96]
 [ -2  -1  -1 ... -93 -93 -94]
 ...
 [-96 -94 -92 ... -48 -49 -50]
 [-97 -95 -93 ... -48 -49 -50]
 [-98 -96 -94 ... -49 -49 -49]]
Best score = -49
[[0 2 2 ... 2 2 2]
 [1 3 2 ... 2 2 2]
 [1 1 3 ... 2 3 2]
 ...
 [1 1 1 ... 2 2 2]
 [1 1 1 ... 3 2 2]
 [1 1 1 ... 1 3 3]]
human        = MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALY_LVCGERG_F_FY_T_PKTRREA_EDLQGS_LQPLA__LEGS__L_QKRGIVEQCCTSICSLY__QLE_NYC_N
human_reverse= NCY_NEL_QYLSCISTCCQEVI__GRKQ_LSGE_L__ALPQLS_G_QLDEAERRTKP_TYFFGRE___GCVLY_LAEVL_HSGCLHQNVFAAAPDPGWLALLALLPLLRMWLAM
Best score = -53


**Problem #5:** Find an optimal *global alignment* between the human insulin sequence and the human insulin sequence reversed using the following scoring:

* +1 for a match
* 0 if the peptides are adjacent alphabetically (i.e. 'M' is adjacent to both 'L' and 'N')
* -1 for all other mismatches
* -1 for an indel

Provide the alignment's score and the alignment itself.

Find an alternate optimal alignment for these sequences that uses the same scoring function but is constrained to match the substring 'EAE', which appears exactly once in each sequence and give its score.

---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.